In [1]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\anshl\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from ipywidgets import widgets, VBox, HTML
from IPython.display import display, Markdown


df = pd.read_csv("Candidate_details.csv")


df["Experience"] = df["Experience"].astype(str).str.extract(r'(\d+)').astype(float)
df["Age"] = df["Age"].astype(str).str.extract(r'(\d+)').astype(float)
df["Vote Received"] = (
    df["Vote Received"].astype(str)
    .str.replace(",", "", regex=False)
    .str.extract(r'(\d+)')
    .astype(float)
)

df["Result"] = (df["Win %"] > 50).astype(int)

features = ["Party","Experience","Vote Received","Win Location (District)","Age","Gender","Education"]
X = df[features]
y_reg = df["Win %"]
y_clf = df["Result"]

cat_cols = ["Party","Win Location (District)","Gender","Education"]
num_cols = ["Experience","Vote Received","Age"]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ("num", StandardScaler(), num_cols)
    ]
)

reg_model = Pipeline([("preprocess", preprocess), ("model", RandomForestRegressor())])
clf_model = Pipeline([("preprocess", preprocess), ("model", RandomForestClassifier())])

reg_model.fit(X, y_reg)
clf_model.fit(X, y_clf)


name_input = widgets.Text(
    placeholder="Enter Candidate Name...",
    description="Name:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="400px")
)

predict_btn = widgets.Button(
    description="Predict Outcome",
    button_style='success',
    icon='search'
)

output = widgets.Output()


def predict_candidate(b):
    with output:
        output.clear_output()

        input_name = name_input.value.strip()
        if input_name == "":
            display(Markdown("⚠️ Please enter a candidate name"))
            return

        candidate = df[df["Candidate Name"].str.lower() == input_name.lower()]

        if candidate.empty:
            display(Markdown(f"❌ No record found for **{input_name}**"))
            return
        
        row = candidate.iloc[0]

        input_data = pd.DataFrame({
            "Party":[row["Party"]],
            "Experience":[row["Experience"]],
            "Vote Received":[row["Vote Received"]],
            "Win Location (District)":[row["Win Location (District)"]],
            "Age":[row["Age"]],
            "Gender":[row["Gender"]],
            "Education":[row["Education"]]
        })

        win_percent = reg_model.predict(input_data)[0]
        prediction = clf_model.predict(input_data)[0]
        outcome = "✅ WIN" if prediction==1 else "❌ LOSS"

        display(Markdown(f"""
### 🧑 Candidate Profile
**Name:** {row['Candidate Name']}  
**Party:** {row['Party']}  
**District:** {row['Win Location (District)']}  
**Experience:** {int(row['Experience'])} Years  
**Age:** {int(row['Age'])}  
**Education:** {row['Education']}  
**Gender:** {row['Gender']}  

---

### 🎯 Election Prediction
- **Expected Win %:** `{win_percent:.2f}%`
- **Outcome:** **{outcome}**
"""))

predict_btn.on_click(predict_candidate)

display(VBox([name_input, predict_btn, output]))
